In [ ]:
%run /Utils/Functions/core

In [ ]:
import requests
import concurrent.futures
import time
import sys
import os
import re
from time import sleep, mktime
from uuid import uuid4
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta, timezone
from IPython.display import clear_output

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = '<layer>'
emissor = '<org>'
atividade = '<activity_description>'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

In [ ]:
%sql
use catalog prod;

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]

print(location_landing)

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# dt_inicio_unix = 1546300800

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

In [ ]:
domain = "graph"
subdomain = "facebook"
pageId = "<page_id>"
threads = 10
baseUrl = f"https://{domain}.{subdomain}.com/v17.0/"
token = dbutils.secrets.get('scope-vault-data', 'facebook-pages-api-token')

headers = {'Authorization': f'Bearer {token}'}

print(baseUrl)

In [ ]:
# Pega apenas cargas que possuem valor preenchido no campo "ds_Url" para não trazer tabelas que são geradas a partir de outro evento na landing.
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'facebook_pages'
    and ds_Url is not null
""")

data_param = df_param.collect()

display(df_param)

## SCRIPT PARA OBTER O PAGE_POST_ID e POST_VIDEO_ID

In [ ]:
postId_set = set()
postVideoId_set = set()

urlPostId = f"https://{domain}.{subdomain}.com/{pageId}/feed?fields=admin_creator,can_reply_privately,created_time,is_eligible_for_promotion,is_expired,feed_targeting,full_picture,is_hidden,icon,id,instagram_eligibility,message,parent_id,permalink_url,place,is_popular,privacy,promotable_id,is_published,is_spherical,status_type,story,story_tags,subscribed,targeting,updated_time,video_buying_eligibility"

while True:
    req = requests.get(urlPostId, headers=headers)
    data = req.json()
    
    for dados in data['data']:
        postId_set.add(dados['id'])

        if dados['status_type'] == 'added_video':
            postVideoId_set.add(dados['id'])

    if len(postId_set) == 150:
        break
    else:
        urlPostId = data['paging']['next']

print("Qtde POST_ID =",len(postId_set))
print("Qtde POST_VIDEO_ID =",len(postVideoId_set))

##SCRIPT DE INGESTÃO DE DADOS

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing, headers):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)

        # Landing
        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
        
        start_date = "1672531200"
        end_date = "1677628800"

        # URL para o request
        url = baseUrl + param.ds_Url

        # Cria uma lista vazia para armazenar as respostas de cada request
        result_data = []

        # limite de registros por resposta da api (valor fixo)
        # esse valor deve ser definido de acordo com a documentação da API
        api_limit = 1000

        # número máximo de erros permitodos antes de parar a execução (tentativas)
        max_errors = 10

        # tempo para esperar para tentar novamente em caso de erro (em segundos)
        error_sleep = 30

        if param.id_Parametro_Carga_API in (384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438):
            url = baseUrl + param.ds_Url.replace('@START_DATE', vlUltimoIncremento).replace('@END_DATE', vlUltimoIncremento).replace('@PAGE_ID', pageId)
            while True:
                response = requests.get(url, headers=headers)
                data = response.json()

                for dados in data['data']:
                    if 'id' in dados.keys():
                        id_tratado = dados['id'].split('/')[0]
                        dados['id'] = id_tratado
            
                    value_list = []
                    for valores in dados['values']:
                        if isinstance(valores['value'], dict):
                            for category, valor in valores['value'].items():
                                value_list.append({"category": category, "value":valor, "end_time":valores["end_time"]})
                                dados["values"] = value_list

                for dados in data['data']:
                    result_data.append(dados)
                
                if 'next' in data['paging'].keys():
                    url = data['paging']['next']
                else:
                    break

        elif param.nm_Item_Origem in 'post':
            url = baseUrl + param.ds_Url.replace('@START_DATE', vlUltimoIncremento).replace('@END_DATE', vlUltimoIncremento).replace('@PAGE_ID', pageId)

            while True:
                req = requests.get(url, headers=headers)
                data = req.json()

                for dados in data['data']:
                    dados['page_id'] = '794115467360694'
                    result_data.append(dados)

                if 'paging' not in data.keys():
                    break
                else:
                    if 'next' in data['paging'].keys():
                        url = data['paging']['next']
                    else:
                        break

        elif param.nm_Item_Origem in 'page':
            url = baseUrl + param.ds_Url.replace('@START_DATE', vlUltimoIncremento).replace('@END_DATE', vlUltimoIncremento).replace('@PAGE_ID', pageId)
            req = requests.get(url=url, headers=headers)
            data = req.json()

            for key, value in data.items():
                if isinstance(value, list):
                    for item in value:
                        data[key] = item
            result_data.append(data)

        elif param.id_Parametro_Carga_API in (439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497):

            for postId in postId_set:
                url = baseUrl + param.ds_Url.replace('@START_DATE', vlUltimoIncremento).replace('@END_DATE', vlUltimoIncremento).replace('@PAGE_ID', pageId).replace('@PAGE_POST_ID', postId)
                req = requests.get(url, headers=headers)

                if req.status_code == 200:
                    data = req.json()

                    for dados in data['data']:
                        if 'id' in dados.keys():
                            id_tratado = dados['id'].split('/')[0]
                            dados['id'] = id_tratado

                        value_list = []
                        for valores in dados['values']:
                            if isinstance(valores['value'], dict):
                                for category, valor in valores['value'].items():
                                    value_list.append({"category": category, "value":valor})
                                    dados["values"] = value_list

                    for dados in data['data']:
                        result_data.append(dados)

                else:
                    print(f"ERRO tabela {param.nm_Item_Origem}, URL: {url}")

        elif param.nm_Item_Origem in 'post_attachment':
            for postId in postId_set:

                url = baseUrl + param.ds_Url.replace('@PAGE_POST_ID', postId)

                req = requests.get(url, headers=headers)
                data = req.json()

                for dados in data['data']:
                    dados['page_post_id'] = f"794115467360694_{dados['target']['id']}"

                for dados in data['data']:
                    result_data.append(dados)
                
        elif param.nm_Item_Origem in 'post_tagged_object':
            while True:
                url = baseUrl + param.ds_Url.replace('@PAGE_ID', pageId)
                req = requests.get(url, headers=headers)
                data = req.json()

                for dados in data['data']:
                    result_data.append(dados)

                if 'next' not in data.keys():
                    break
                else:
                    url = data['next']

        elif param.nm_Item_Origem in 'post_videos':
            for postVideoId in postVideoId_set:
                url = baseUrl + param.ds_Url.replace('@POST_VIDEO_ID', postVideoId)
                req = requests.get(url, headers=headers)

                if req.status_code == 200:
                    data = req.json()
                    result_data.append(data)

                else:
                    print(f"ERRO tabela {param.nm_Item_Origem}, URL: {url}")
        
        if len(result_data) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
            return

        fn_SaveJson(result_data, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())
        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)
    
        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

        return f"Carga da tabela {param.ds_Nome_Arquivo_Landing} finalizada com sucesso."

    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    # Percorre todas as pastas do diretório de origem
    for row in data_param:
        # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
        task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing, headers))
                               
        # Adiciona a tarefa à lista de tarefas
        tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

In [ ]:
for task in tasks:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass